# Exp013: Create prompts for generating grammar-controlled text

In [6]:
import random
import os
import sys
sys.path.append('../source')
import models
import data
import api
import importlib
#importlib.reload(api)

<module 'api' from '/cluster/home/dglandorf/grammarctg/experiments/../source/api.py'>

Load EGP rules and topical prompts 

In [117]:
egp = data.get_egp()
cefr = data.CEFRTexts()
stories = list(cefr.get_beginnings(100))
levels = ["A1", "A2", "B1", "B2", "C1", "C2"] 

Assemble prompt

In [121]:
def skills(rules, level):
    level_rules = rules[rules['Level'] == level]
    return "- " + subcategory + " - " + level_rules['guideword'] + ": " + level_rules['Can-do statement']

In [136]:
subcategory = "would"
rules = egp[(egp['SubCategory']==subcategory)]
level = 3
prompt = f"""
Continue the text proving knowledge of at least one of these grammar skills:
{os.linesep.join(skills(rules, levels[level-1]))}
Do NOT apply the following grammar skills:
{os.linesep.join(skills(rules, levels[level]))}

Text. {random.sample(stories, 1)[0]}
"""
print(prompt)


Continue the text proving knowledge of at least one of these grammar skills:
- would - FORM/USE: AFTER 'IF' CLAUSES: Can use 'would' in the main clause of a conditional sentence to talk about an imagined situation, often in the context of advice or opinion-giving.
- would - FORM: PAST AFFIRMATIVE: Can use 'would have' + '-ed'.
- would - FORM: PAST NEGATIVE: Can use 'would not have' + '-ed' or 'wouldn’t have' + '-ed'
- would - FORM: QUESTIONS: Can use question forms.
- would - FORM: WITH ADVERBS: Can use an limited range of adverbs with 'would', including 'really', 'probably', 'certainly', 'definitely'.► adverbs
- would - USE: FUTURE IN THE PAST: Can use 'would' to talk about the future in the past.
- would - USE: IMAGINED SITUATIONS IN THE PAST: Can use 'would' to talk about imagined situations in the past. ► conditionals
- would - USE: INDIRECTNESS: Can use 'would' with verbs such as 'advise', 'imagine', 'recommend', 'say' to be less direct.
- would - USE: POLITE REQUESTS: Can use 'w

Call GPT3.5

In [138]:
messages = [{ "role": "user", "content": prompt}]
response = api.get_openai_chat_completion(messages)[0]
init_sent = data.sent_tokenize(prompt)[-1]
sentences = [(init_sent + " " if not init_sent[-1]=="." else "") + data.sent_tokenize(response)[0].strip()] + \
data.sent_tokenize(response)[1:]
print(sentences)

['She just wanted to get home, order a pizza for dinner, and watch a movie.', 'If I were you, I would choose a comedy to unwind and relax."', 'Michelle thought to herself as she boarded the bus home.', 'She imagined herself laughing and forgetting about the stress of the day as she enjoyed a funny movie.', 'The thought of a cheesy joke from the movie made her smile, and she knew it would be the perfect way to end the day.', 'Once she arrived home, she ordered her favorite pizza and selected a comedy from her movie collection.', "As she settled in on the couch with her dinner, she couldn't help but feel grateful for the simple pleasures in life."]


Use classifiers to score the response

In [105]:
classifiers = {nr: models.load_classifier(nr, 'corpus_training') for nr in cat_rules['#']}

In [139]:
for level, skill, (nr, classifier) in zip(rules['Level'], rules['Can-do statement'], classifiers.items()):    
    scores = models.probe_model(classifiers[nr], sentences)
    detected_sentences = [sentences[i] for i, score in enumerate(scores[0]) if score > 0.5]
    if len(detected_sentences): print(f'Detected Level {nr}-{level}-{skill}: {detected_sentences}')

Detected Level 619-A2-Can use the affirmative form.: ['If I were you, I would choose a comedy to unwind and relax."', 'The thought of a cheesy joke from the movie made her smile, and she knew it would be the perfect way to end the day.']
Detected Level 622-A2-Can use 'would' to talk about imagined situations.: ['The thought of a cheesy joke from the movie made her smile, and she knew it would be the perfect way to end the day.']
Detected Level 625-B1-Can use 'would' in the main clause of a conditional sentence to talk about an imagined situation, often in the context of advice or opinion-giving.: ['If I were you, I would choose a comedy to unwind and relax."']
Detected Level 630-B1-Can use 'would' to talk about the future in the past.: ['The thought of a cheesy joke from the movie made her smile, and she knew it would be the perfect way to end the day.']
Detected Level 632-B1-Can use 'would' with verbs such as 'advise', 'imagine', 'recommend', 'say' to be less direct.: ['If I were you,